In [1]:
import pandas as pd
import numpy as np

In [2]:
# Start writing code here...import numpy as np
def write_distance_matrix(n, mean, sigma):
    distance_matrix = np.zeros((n, n))

    for row in range(n):
        for col in range(n):
            distance = 0
            while distance <= 0:
                distance = np.random.normal(mean, sigma)
                distance_matrix[row][col] = distance

    np.savetxt(
        f"{n}_{mean}_{sigma}.out",
        distance_matrix,
        delimiter=" ",
        fmt="%1.4f",
        header=str(n),
        comments="",
    )


# if __name__ == "__main__":
#     n = int(input("Enter the number of locations: "))
#     mean = float(input("Enter the mean: "))
#     sigma = float(input("Enter the standard deviation: "))

#     write_distance_matrix(n, mean, sigma)


## Read Input

In [3]:
with open('10_0.0_1.0.out', 'r') as file:
    lines = file.readlines()

m = [line.split(' ') for line in lines[1:]]
m = [list(map(lambda x:np.float32(x.strip('\n')), i)) for i in m]
m = np.matrix(m)
np.fill_diagonal(m, 0)
m


matrix([[0.    , 0.1048, 0.032 , 0.7036, 0.0578, 1.6942, 0.0964, 0.2619,
         0.234 , 0.5597],
        [0.5644, 0.    , 0.8384, 0.6193, 1.6502, 0.7474, 0.1451, 1.3065,
         1.6627, 1.2185],
        [1.046 , 0.8041, 0.    , 0.2652, 1.7696, 1.1906, 0.9743, 0.5719,
         1.6883, 1.505 ],
        [0.6046, 0.8627, 0.9758, 0.    , 2.2477, 0.2166, 0.5113, 0.2156,
         1.4176, 0.6256],
        [0.4792, 0.7802, 0.6515, 2.1705, 0.    , 0.8215, 2.8714, 0.3657,
         1.3732, 0.4379],
        [0.5777, 0.4763, 0.2799, 0.365 , 1.2971, 0.    , 0.3548, 0.2826,
         0.7593, 1.3932],
        [0.131 , 0.0288, 0.3833, 0.9969, 0.5015, 0.4838, 0.    , 0.938 ,
         0.44  , 0.7248],
        [1.5668, 0.713 , 0.5895, 0.1167, 1.4093, 0.9177, 0.7382, 0.    ,
         1.5902, 0.9281],
        [0.1835, 2.1879, 0.2694, 0.7045, 1.5368, 1.7719, 0.5463, 1.4507,
         0.    , 1.0938],
        [0.6835, 1.0318, 0.0277, 0.1915, 0.2952, 0.396 , 0.2791, 0.7307,
         1.1793, 0.    ]], dtype=flo

## Branch-and-Bound Back-tracking Depth-First Search

In [4]:
import numpy as np
import math
import time


class BranchAndBound:
    def __init__(self, matrix):  
    
        self.matrix = matrix
        self.min_edge = np.where(matrix>0, matrix, np.inf).min()

        self.num_towns = len(matrix)
        self.towns = list(range(len(matrix))) # Ex: numTowns = 3 →  [0, 1,  2] 

        self.best_path = None
        self.best_cost = math.inf

    def isComplete(self, path): 
        """
        Checks if our current path is a complete path (returns back to the start node !!!)
        """
        if len(path) == self.num_towns + 1:
            return True

    def heuristic(self, path): 
        """
        Heuristic to estimate cost from current path node to goal
        """
        # num_unvisited should include edge back to the start node, must add 1 to num_towns
        # Ex: num_towns = 3; heuristic cost of a complete path (from last node which is start node to the "end") = 0 * min_edge = 0
        num_unvisited =  (self.num_towns + 1) - len(path) 
        h_n = num_unvisited * self.min_edge
        return h_n

    
    
    def computeCost(self, path): #returns (float) 
        """
        Calculates total cost of the path so far 
        """
        cost = 0   
        for i in range(len(path) - 1):
            cost += self.matrix[path[i], path[i+1]]
        return cost


    def print_path(self):
        print(" → ".join([str(i) for i in self.best_path]))
        return


    def branchAndBound(self):
        start_time = time.time()
        root = 0  # Assume starting town is always 0
        stack = []
        stack.append([root])

        while len(stack) != 0:
                current = stack.pop() # remove and retrieve last item put into the stack

                if self.computeCost(current) + self.heuristic(current) < self.best_cost:
                    if self.isComplete(current): # complete if returns back to the start node !!!
                        self.best_path = current
                        self.best_cost = self.computeCost(current)
                    else: # if not a complete path
                        if len(current) < self.num_towns:
                            for t in self.towns:
                                if t not in current:
                                    new = current + [t]
                                    stack.append(new) 
                        elif len(current) == self.num_towns:
                            new = current + [0]
                            stack.append(new)

        # print best path
        print("--------------------------------------------------------------------")
        print("Ending cost: {}".format(float(self.best_cost)))
        print("Ending path:")
        self.print_path()                         
        print("--------------------------------------------------------------------")
        print("Total time taken for {} cities: {} seconds".format(self.num_towns, time.time()-start_time))
        print("--------------------------------------------------------------------")
        print("Solution with path cost {}".format(self.best_cost))
  

        return


def read_cities_from_file(filename = "10_0.0_1.0.out"):
    
    """
    Read in distance matrix from file, clean and assign to BnB object.
    """
    
    with open(filename, 'r') as file:
        lines = file.readlines()
    m = [line.split(' ') for line in lines[1:]]
    m = [list(map(lambda x:np.float32(x.strip('\n')), i)) for i in m]
    m = np.matrix(m)
    np.fill_diagonal(m, 0)
    
    return np.squeeze(np.asarray(m))


In [ ]:
test_5_cities_m = read_cities_from_file('5_0.0_5.0.out')
test_5_cities = BranchAndBound(test_5_cities_m) # create BnB class instance
test_5_cities.branchAndBound()

--------------------------------------------------------------------
Ending cost: 9.967799857258797
Ending path:
0 → 2 → 4 → 3 → 1 → 0
--------------------------------------------------------------------
Total time taken for 5 cities: 0.000499725341796875 seconds
--------------------------------------------------------------------
Solution with path cost 9.967799857258797


In [ ]:
test_10_cities_m = read_cities_from_file('10_0.0_1.0.out')
test_10_cities = BranchAndBound(test_10_cities_m) # create BnB class instance
test_10_cities.branchAndBound()

--------------------------------------------------------------------
Ending cost: 2.673500020056963
Ending path:
0 → 4 → 9 → 2 → 7 → 3 → 5 → 1 → 6 → 8 → 0
--------------------------------------------------------------------
Total time taken for 10 cities: 0.46094298362731934 seconds
--------------------------------------------------------------------
Solution with path cost 2.673500020056963


In [ ]:
test_50_cities_m = read_cities_from_file('50_0.0_1.0.out')
test_50_cities = BranchAndBound(test_50_cities_m) # create BnB class instance
test_50_cities.branchAndBound()

KeyboardInterrupt: 

## Stochastic Local Search

To-do: 

- Function to handle whether or not returning to start node

- Test out different annealing strategies w/ graphs to add to report: i.e. max changes, temperature, max paths (average results)

- Fix dict to arr (done)

- Python exe

- Improve docstrings/comments

In [ ]:
import numpy as np
import pandas as pd
import string
import math
import random
import time

class LocalSearch:
    def __init__(self, matrix, return_to_start=False):
        
        if return_to_start:
            self.return_to_start = True
            
        self.temperature = 0.5 #Initial annealing temperature
        self.num_cities = len(matrix)
        self.max_changes = (self.num_cities*100)/4
        self.max_paths = self.num_cities*100
        self.current_best = []
        self.current_best_distance = math.inf
        self.cities = matrix 
        self.return_to_start = False
        
    def metropolis(self, cost_difference): 
        """
        Compute the metropolis criterion with parameters cost_difference (int) and global temperature T (float). 
        Returns bool.
        """
        return cost_difference < 0 or (random.choice([1,0]) < np.e**(-cost_difference/self.temperature))
    
    def random_select(self):
    
        """
        Randomly select 2 cities, return 2 different cities.
        """
        
        temp_cities = [i for i in range(self.num_cities)]
    
        city1 = random.choice(temp_cities)
        temp_cities.remove(city1)
    
        city2 = random.choice(temp_cities)
    
        c1_idx = self.current_best.index(city1)
        c2_idx = self.current_best.index(city2)
        
        return city1, city2, c1_idx, c2_idx
    
    def swap(self, proposed_path, new_cost):

        """
        Swap paths.
        """

        self.current_best_distance = new_cost
        self.current_best = proposed_path
        
        return
    
    def generate_temperature(self):
        """
        Recompute temperature.
        """
        self.temperature*=0.9
        return 

    def print_path(self):
        print(" → ".join([str(i) for i in self.current_best]))
        return
    
    def compute_cost(self, path):
    
        cost = 0
        for i in range(len(path)-1):
            cost+=self.cities[path[i]][path[i+1]]
        
        return cost
    
    def initialize_path(self, starting_city=None):
        """
        Creates initial random path for annealing process. Returns a randomized path.
        """
        
        temp_cities = [i for i in range(self.num_cities)]
    
        path = []
    
        for _ in range(self.num_cities):
            city = random.choice(temp_cities)
            path.append(city)
            temp_cities.remove(city)
        
        cost = self.compute_cost(path)
        
        self.current_best = path
        self.current_best_distance = cost
        
        return 
    
        
    def localSearch(self, starting_city=None):
        
        start_time = time.time()
        self.initialize_path(starting_city=starting_city)
        num_iterations = 0
        print("Starting cost: {}".format(float(self.current_best_distance)))
        print("Starting path:")
        self.print_path()
        
        while self.temperature > 0:
            
            num_changes = 0 
            for _ in range(self.max_paths):
                
                proposed_path = self.current_best
                
                city1, city2, c1_idx, c2_idx = self.random_select()
                
                proposed_path[c1_idx], proposed_path[c2_idx] = city2, city1
                
                new_cost = self.compute_cost(proposed_path)
                
                cost_difference = new_cost - self.current_best_distance
                
                if self.metropolis(cost_difference):
                    self.swap(proposed_path, new_cost)
                    num_changes+=1
                    if num_changes==self.max_changes:
                        break            
            if num_changes==0:
                break
            num_iterations+=1
            self.generate_temperature()
        
        print("--------------------------------------------------------------------")
        print("Ending cost: {}".format(float(self.current_best_distance)))
        print("Ending path:")
        self.print_path()
        print("--------------------------------------------------------------------")
        print("Total iterations: {}".format(num_iterations))
        print("Total time taken for {} cities: {} seconds".format(self.num_cities, time.time()-start_time))
        print("--------------------------------------------------------------------")
        print("Solution with path cost {} at temperature {}".format(self.current_best_distance, self.temperature))

        return
    
def read_cities_from_file(filename = "10_0.0_1.0.out"):
    
    """
    Read in distance matrix from file, clean and assign to LocalSearch object.
    """
    
    with open(filename, 'r') as file:
        lines = file.readlines()
    m = [line.split(' ') for line in lines[1:]]
    m = [list(map(lambda x:np.float32(x.strip('\n')), i)) for i in m]
    m = np.matrix(m)
    np.fill_diagonal(m, 0)
    
    return np.squeeze(np.asarray(m))

In [ ]:
test_5_cities_m = read_cities_from_file("5_0.0_5.0.out")
test_5_cities = LocalSearch(test_5_cities_m)
test_5_cities.localSearch()


Starting cost: 23.000699996948242
Starting path:
4 → 2 → 0 → 1 → 3
--------------------------------------------------------------------
Ending cost: 6.375299885869026
Ending path:
2 → 0 → 1 → 3 → 4
--------------------------------------------------------------------
Total iterations: 74
Total time taken for 5 cities: 0.3611123561859131 seconds
--------------------------------------------------------------------
Solution with path cost 6.375299885869026 at temperature 0.00020554915835284865


In [ ]:
test_10_cities_m = read_cities_from_file()
test_10_cities = LocalSearch(test_10_cities_m)
test_10_cities.localSearch()


Starting cost: 7.625600155442953
Starting path:
1 → 8 → 4 → 7 → 6 → 3 → 5 → 9 → 0 → 2
--------------------------------------------------------------------
Ending cost: 3.2629000172019005
Ending path:
5 → 2 → 3 → 8 → 7 → 6 → 1 → 0 → 4 → 9
--------------------------------------------------------------------
Total iterations: 67
Total time taken for 10 cities: 0.509068489074707 seconds
--------------------------------------------------------------------
Solution with path cost 3.2629000172019005 at temperature 0.00042975222785857195


In [ ]:
test_100_cities_m = read_cities_from_file('100_25.0_5.0.out')
test_100_cities = LocalSearch(test_100_cities_m)
test_100_cities.localSearch()


Starting cost: 2540.1410026550293
Starting path:
94 → 84 → 63 → 91 → 54 → 53 → 1 → 77 → 30 → 76 → 83 → 20 → 26 → 10 → 86 → 58 → 14 → 65 → 95 → 64 → 40 → 72 → 18 → 78 → 21 → 31 → 35 → 22 → 90 → 51 → 79 → 39 → 99 → 50 → 55 → 88 → 6 → 2 → 3 → 34 → 43 → 25 → 5 → 28 → 70 → 41 → 74 → 96 → 0 → 81 → 57 → 97 → 92 → 32 → 45 → 12 → 46 → 27 → 66 → 68 → 56 → 24 → 62 → 98 → 4 → 52 → 38 → 7 → 73 → 44 → 75 → 47 → 8 → 85 → 71 → 29 → 67 → 61 → 89 → 60 → 87 → 16 → 80 → 33 → 93 → 17 → 69 → 15 → 37 → 48 → 82 → 9 → 36 → 13 → 49 → 59 → 11 → 23 → 19 → 42
--------------------------------------------------------------------
Ending cost: 2313.7168016433716
Ending path:
53 → 54 → 72 → 3 → 86 → 11 → 12 → 29 → 8 → 60 → 76 → 90 → 14 → 94 → 74 → 75 → 63 → 20 → 98 → 55 → 9 → 4 → 17 → 95 → 56 → 13 → 99 → 85 → 23 → 92 → 38 → 22 → 57 → 82 → 19 → 91 → 16 → 49 → 70 → 96 → 69 → 44 → 31 → 67 → 71 → 25 → 42 → 34 → 51 → 59 → 77 → 50 → 28 → 18 → 45 → 40 → 84 → 41 → 68 → 73 → 66 → 10 → 64 → 39 → 46 → 52 → 24 → 62 → 87 → 78 → 47 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ee05829a-7cbd-4d9e-807e-4b97fdc932b0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>